In [280]:
import pandas
data_folder = "/home/aitor/Escritorio/TFM/Datos"
data_name = "data.csv"
data_path = "/".join((data_folder, data_name))
data = pandas.read_csv(data_path)
data_transpose = data.transpose()
data_by_questions = data_transpose.iloc[8:]

In [281]:
import json
questions_name = "questions.json"
questions_path = "/".join((data_folder, questions_name))
with open(questions_path) as json_file:
    questions = json.load(json_file)

In [282]:
def transform_table(original_table, questions_template):
    new_table = original_table.copy()
    for question, values in new_table.iterrows():
        # Check if the question is in the 'questions' dict
        if question in questions:
            #print("Respuesta correcta: {}".format(questions[question]["correct_answer"]))
            for index, value in values.iteritems():
                if value != "NS/NC":
                    new_table.at[question, index] = (value == questions_template[question]["correct_answer"])
        else:
            print("Error!")
            
    return new_table

In [283]:
# Add 3 columns at the end with Correct (True), Incorrent (False) and NS/NC (NS/NC) values.
# Only valid for this three possible values
def add_results_columns(original_table):
    new_table = original_table.copy()
    true_list = []
    false_list = []
    nsnc_list = []
    for question, values in new_table.iterrows():
        true_counter = 0
        false_counter = 0
        nsnc_counter = 0
        for index, value in values.iteritems():
            if value is True:
                true_counter += 1
            elif value is False:
                false_counter += 1
            elif value == "NS/NC":
                nsnc_counter +=1
        true_list.append(true_counter)
        false_list.append(false_counter)
        nsnc_list.append(nsnc_counter)

    true_series = pandas.Series(true_list, index=new_table.index)
    false_series = pandas.Series(false_list, index=new_table.index) 
    nsnc_series = pandas.Series(nsnc_list, index=new_table.index)
    
    new_table["Correct"] = true_series
    new_table["Incorrect"] = false_series
    new_table["NS/NC"] = nsnc_series
    
    return new_table

In [284]:
def delete_non_valid_columns(original_table, valid_list):
    return original_table.filter(valid_list)

In [285]:
def get_table_info(original_table, questions):
    column_list = ["Correct", "Incorrect", "NS/NC"]
    return delete_non_valid_columns(add_results_columns(transform_table(original_table, questions)), column_list)
    
    

In [286]:
all_questions_info = get_table_info(data_by_questions, questions)

In [287]:
def split_table_by_myth(original_table, questions_template):
    myth_list = [question for question, info in questions_template.items() if info["myth"]]
    non_myth_list = [question for question, info in questions_template.items() if not info["myth"]]
    
    myth_table = original_table.filter(myth_list)
    non_myth_table = original_table.filter(non_myth_list)
    
    return (myth_table, non_myth_table)

In [288]:
myth_table, non_myth_table = split_table_by_myth(data, questions)

In [289]:
myth_info = get_table_info(myth_table.transpose(), questions)
non_myth_info = get_table_info(non_myth_table.transpose(), questions)

In [290]:
#non_myth_info
#myth_info

In [291]:
def get_table_info_percentage(original_table, delete=False):
    new_table = original_table.copy()
    correct_porcentage_list = []
    incorrect_porcentage_list = []
    nsnc_porcentage_list = []
    for question, values in new_table.iterrows():
        total = values.sum()
        correct_porcentage_list.append(round(values["Correct"]*100/total,1))
        incorrect_porcentage_list.append(round(values["Incorrect"]*100/total,1))
        nsnc_porcentage_list.append(round(values["NS/NC"]*100/total))

    correct_porcentage_series = pandas.Series(correct_porcentage_list, index=new_table.index)
    incorrect_porcentage_series = pandas.Series(incorrect_porcentage_list, index=new_table.index) 
    nsnc_porcentage_series = pandas.Series(nsnc_porcentage_list, index=new_table.index)
    
    new_table["Correct (%)"] = correct_porcentage_series
    new_table["Incorrect (%)"] = incorrect_porcentage_series
    new_table["NS/NC (%)"] = nsnc_porcentage_series
    
    if delete:
        new_table = new_table.drop(["Correct", "Incorrect", "NS/NC"], axis=1)
    
    return new_table

In [292]:
myth_info_porcentage = get_table_info_percentage(myth_info, delete=True)

In [293]:
non_myth_info_porcentage = get_table_info_percentage(non_myth_info, delete=True)

In [294]:
myth_info_porcentage.sort_values("Incorrect (%)", ascending=False)

,Correct (%),Incorrect (%),NS/NC (%)
"11. Las personas aprenden mejor cuando reciben la información en su estilo de aprendizaje preferido (ej. auditivo, visual y cinestésico).",7.0,86.0,7.0
13. Los entornos que son ricos en estímulos mejoran los cerebros de los niños y niñas preescolares.,7.0,82.6,10.0
14. Los ejercicios que promueven la coordinación de las habilidades perceptivo-motoras pueden mejorar las destrezas en lecto-escritura.,2.9,76.2,21.0
"7. Las diferencias en el hemisferio dominante (cerebro izquierdo, cerebro derecho) pueden ayudar a explicar las diferencias individuales entre estudiantes.",11.0,47.1,42.0
5. Solo usamos el 10% del cerebro.,44.8,37.2,18.0
10. Hay períodos críticos en la infancia después de los cuales ciertas cosas ya no pueden ser aprendidas.,69.2,17.4,13.0
"2. Los niños deben adquirir su lengua materna antes de aprender una segunda lengua. Si no lo hacen así, ninguna de las dos lenguas será adquirida por completo.",73.8,11.6,15.0
17. Los problemas de aprendizaje asociados con diferencias de desarrollo en la función cerebral no pueden remediarse mediante la educación.,64.0,5.8,30.0


In [295]:
non_myth_info_porcentage.sort_values("Correct (%)", ascending=False)

,Correct (%),Incorrect (%),NS/NC (%)
25. Aprender dos idiomas supone una ventaja cognitiva.,91.9,1.7,6.0
"16. Cada alumno muestra una preferencia por el modo en que recibe la información (ej. visual, auditivo, cinestésico).",86.0,2.9,11.0
20. La inteligencia puede variar con el ambiente o la experiencia.,83.1,7.0,10.0
1. Usamos el cerebro 24 horas al día.,82.0,10.5,8.0
18. La producción de nuevas conexiones en el cerebro puede continuar hasta la vejez.,77.3,4.7,18.0
9. El desarrollo del cerebro finaliza para cuando los niños y niñas llegan a secundaria.,72.1,3.5,24.0
21. La inteligencia es única.,69.8,9.3,21.0
22. Los genes influyen en la personalidad de los estudiantes.,66.3,11.6,22.0
15. La práctica repetida de algunos procesos mentales puede cambiar la forma y la estructura de algunas partes del cerebro.,57.0,9.3,34.0
19. La inteligencia tiene una componente hereditaria.,54.1,22.7,23.0


In [296]:
myth_correct_mean = myth_info_porcentage["Correct (%)"].mean()
myth_correct_std = myth_info_porcentage["Correct (%)"].std()
myth_incorrect_mean = myth_info_porcentage["Incorrect (%)"].mean()
myth_incorrect_std = myth_info_porcentage["Incorrect (%)"].std()
myth_nsnc_mean = myth_info_porcentage["NS/NC (%)"].mean()
myth_nsnc_std = myth_info_porcentage["NS/NC (%)"].std()
print("Media correctos: {:.1f} (SD: {:.1f})".format(myth_correct_mean, myth_correct_std))
print("Media incorrectos: {:.1f} (SD: {:.1f})".format(myth_incorrect_mean, myth_incorrect_std))
print("Media ns/nc: {:.1f} (SD: {:.1f})".format(myth_nsnc_mean, myth_nsnc_std))

Media correctos: 35.0 (SD: 31.1)
Media incorrectos: 45.5 (SD: 32.8)
Media ns/nc: 19.5 (SD: 11.5)


In [297]:
non_myth_correct_mean = non_myth_info_porcentage["Correct (%)"].mean()
non_myth_correct_std = non_myth_info_porcentage["Correct (%)"].std()
non_myth_incorrect_mean = non_myth_info_porcentage["Incorrect (%)"].mean()
non_myth_incorrect_std = non_myth_info_porcentage["Incorrect (%)"].std()
non_myth_nsnc_mean = non_myth_info_porcentage["NS/NC (%)"].mean()
non_myth_nsnc_std = non_myth_info_porcentage["NS/NC (%)"].std()
print("Media correctos: {:.1f} (SD: {:.1f})".format(non_myth_correct_mean, non_myth_correct_std))
print("Media incorrectos: {:.1f} (SD: {:.1f})".format(non_myth_incorrect_mean, non_myth_incorrect_std))
print("Media ns/nc: {:.1f} (SD: {:.1f})".format(non_myth_nsnc_mean, non_myth_nsnc_std))

Media correctos: 56.5 (SD: 25.9)
Media incorrectos: 20.3 (SD: 19.9)
Media ns/nc: 23.1 (SD: 10.1)
